In [ ]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from torch import nn
from torch.nn import BCELoss
from torch.optim import SGD, Adam
from keras_preprocessing.text import Tokenizer 
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tqdm import tqdm
import tqdm
import time

In [ ]:
class TextCodeDataset(Dataset):
    def __init__(self, data_path, pad_seq_len, use_AST=True, classify_btype=True):
        self.data_path = data_path
        self.pad_seq_len = pad_seq_len
        self.use_AST, self.classify_btype = use_AST, classify_btype
        self.data_pd = pd.read_csv(data_path,
                                   error_bad_lines=False, index_col=False, dtype='unicode', encoding='latin-1',
                                   low_memory=False).sample(frac=1)
        self.y_dev, self.y_btype = list(self.data_pd['FixedByID']), list(self.data_pd['Name'])
        self.y_dev, self.y_btype = [str(x) for x in self.y_dev], [str(x) for x in self.y_btype]
        self.y_dev, self.y_btype = [x.split('|') for x in self.y_dev], [x.split('|') for x in self.y_btype]

        self.x_context, self.x_AST = list(self.data_pd['Title_Description']), list(self.data_pd['AST'])
        self.x_context, self.x_AST = [str(x) for x in self.x_context], [str(x) for x in self.x_AST]

    def tokenize_input(self, tokenizer_C: Tokenizer, tokenizer_A: Tokenizer):
        self.x_context = tokenizer_C.texts_to_sequences(self.x_context)
        self.x_AST = tokenizer_A.texts_to_sequences(self.x_AST)
        self.x_context = pad_sequences(self.x_context, maxlen=self.pad_seq_len, padding='post')
        self.x_AST = pad_sequences(self.x_AST, maxlen=self.pad_seq_len, padding='post')
        self.x_context = torch.from_numpy(self.x_context)
        self.x_AST = torch.from_numpy(self.x_AST)

    def map_output(self, map_d: dict, map_b: dict):
        tensor_d, tensor_b = torch.zeros((len(self.data_pd), len(map_d))), torch.zeros((len(self.data_pd), len(map_b)))
        for i, ds in enumerate(self.y_dev):
            for d in ds:
                tensor_d[i][map_d[d]] = 1
        for i, bs in enumerate(self.y_btype):
            for b in bs:
                tensor_b[i][map_b[b]] = 1
        self.y_dev, self.y_btype = tensor_d, tensor_b

    def __getitem__(self, i):
        input = self.x_context[i], self.x_AST[i] if self.use_AST else self.x_context[i]
        output = self.y_dev[i], self.y_btype[i] if self.classify_btype else self.y_dev[i]
        return input, output

    def __len__(self):
        return len(self.data_pd)

In [ ]:
def tokenize_dataset_input(train_dataset: TextCodeDataset, test_dataset: TextCodeDataset):
    tokenizer_C = Tokenizer()
    tokenizer_C.fit_on_texts(train_dataset.x_context + test_dataset.x_context)
    tokenizer_A = Tokenizer()
    tokenizer_A.fit_on_texts(train_dataset.x_AST + test_dataset.x_AST)
    train_dataset.tokenize_input(tokenizer_C, tokenizer_A)
    test_dataset.tokenize_input(tokenizer_C, tokenizer_A)
    return [len(tokenizer_C.word_index)+100, len(tokenizer_A.word_index)+100]


def map_dataset_output(train_dataset: TextCodeDataset, test_dataset: TextCodeDataset):
    set_d, set_b = set(), set()
    for i in range(len(train_dataset)):
        for d in train_dataset.y_dev:
            set_d.update(d)
        for b in train_dataset.y_btype:
            set_b.update(b)
    for i in range(len(test_dataset)):
        for d in test_dataset.y_dev:
            set_d.update(d)
        for b in test_dataset.y_btype:
            set_b.update(b)
    labels_d, labels_b = list(set_d), list(set_b)
    labels_d.sort(), labels_b.sort()
    map_d, map_b = {labels_d[i]: i for i in range(len(labels_d))}, {labels_b[i]: i for i in range(len(labels_b))}
    train_dataset.map_output(map_d, map_b)
    test_dataset.map_output(map_d, map_b)
    return labels_d, labels_b

In [ ]:
# dataset
# train_path = 'Data/powershell/C_uA_Train.csv'
train_path = '../Data/aspnet/trainC40.csv'
test_path = '../Data/aspnet/Nov10数据/IssueaspnetcoreWebScraptestdata5.csv'
MAX_SEQ_LEN = 300
EMB_DIM = 100
Learning_Rate = 6e-3
EPOCH = 300
device = 'cuda' if torch.cuda.is_available() else 'cpu'
Weight_MSE, Weight_BCE = 0, 1
# 0, 1.0, 1e-3, 100 -> 0.008
# 0.1, 0.9, 1e-3, 100 -> 0.04
# 0.2, 0.8, 1e-3, 100 -> 0.06
# 0.3, 0.7, 1e-3, 100 -> 0.12
# 0.4, 0.6, 1e-3, 100 -> 0.15
# 0.5, 0.5, 1e-3, 100 -> 0.19
# 0.6, 0.4, 1e-3, 100 -> 0.2
# 0.7, 0.3, 1e-3, 100 -> 0.2, stable
# 0.8, 0.2, 1e-3, 100 -> 0.2, convergence   | 0.07 simple model
# 0.9, 0.1, 1e-3, 100 -> 0.15, convergence
# 1, 0, 1e-3, 100 -> ~0
train_dataset = TextCodeDataset(train_path, pad_seq_len=MAX_SEQ_LEN)
test_dataset = TextCodeDataset(test_path, pad_seq_len=MAX_SEQ_LEN)

vocab_size = tokenize_dataset_input(train_dataset, test_dataset)
idx2label = map_dataset_output(train_dataset, test_dataset)
num_out = [len(x) for x in idx2label]

# dataloader
train_loader = DataLoader(train_dataset, batch_size=64)
test_loader = DataLoader(test_dataset, batch_size=64)

In [ ]:
print(len(train_loader))
for data in train_loader:
    (a, b), (c, d) = data
    print(a.shape)
    print(b.shape)
    print(c.shape)
    print(d.shape)
    (a, b), (c, d) = train_dataset[0]
    print(a.shape)
    print(b.shape)
    print(c.shape)
    print(d.shape)
    break

In [ ]:
def testnan(name, x):
    if torch.isnan(x).any():
        print(name, x)

def repnan(x):
    return torch.where(torch.isnan(x), torch.ones_like(x), x)

In [ ]:
#TODO :训练的太快了，是不是参数太少了拟合不了？
# class MetaModel(nn.Module):
#     def __init__(self, vocab_size: list, emb_dim: int, seq_len: int, num_out: list):
#         """
#         Args:
#             vocab_size: list, the size of C/A vocab
#             emb_dim: : int, the dim of C&A emb layer
#             seq_len:  MAX_SEQ_LEN
#             num_out: list, the output size of D/B
#         """
#         super(MetaModel, self).__init__()
#         self.vocab_size = vocab_size
#         self.emb_dim = emb_dim
#         self.seq_len = seq_len
#         self.num_out = num_out

#         self.context = nn.Sequential(
#             nn.Embedding(self.vocab_size[0], self.emb_dim),
#             nn.Conv1d(self.seq_len, 64, kernel_size=2),
#             nn.ReLU(),
#             nn.Conv1d(64, 128, kernel_size=2),
#             nn.ReLU(),
#             nn.MaxPool1d(50, 1),
#             nn.Flatten(),
#         )
#         self.AST = nn.Sequential(
#             nn.Embedding(self.vocab_size[1], self.emb_dim),
#             nn.Conv1d(self.seq_len, 50, kernel_size=2),
#             nn.ReLU(),
#             nn.Conv1d(50, 100, kernel_size=2),
#             nn.ReLU(),
#             nn.MaxPool1d(50, 1),
#             nn.Flatten(),
#         )
#         self.fc = nn.Sequential(
#             nn.BatchNorm1d(11172, affine=False),
#             nn.Dropout(0.5),
#             nn.Linear(11172, 5000),
#             nn.ReLU(),
#             nn.Dropout(0.5),
#             nn.Linear(5000, 1000),
#             nn.ReLU(),
#             nn.Dropout(0.5),
#             nn.Linear(1000, 500),
#             nn.ReLU(),
#         )
#         self.fc_d = nn.Sequential(
#             nn.Linear(500, self.num_out[0]),
#             # nn.Softmax(dim=1),
#             # nn.Sigmoid(),
#         )

#         self.fc_b = nn.Sequential(
#             nn.Linear(500, self.num_out[1]),
#             # nn.Softmax(dim=1),
#             # nn.Sigmoid(),
#         )

#     def forward(self, x_C, x_A):
#         # x, y
#         x_C = self.context(x_C) 
#         x_A = self.AST(x_A)
#         x = torch.concat((x_C, x_A), 1)
#         x = self.fc(x)
#         y_d = self.fc_d(x) 
#         y_b = self.fc_b(x) 
#         return y_d, y_b 
class MetaModel(nn.Module):
    def __init__(self, vocab_size: list, emb_dim: int, seq_len: int, num_out: list):
        """
        Args:
            vocab_size: list, the size of C/A vocab
            emb_dim: : int, the dim of C&A emb layer
            seq_len:  MAX_SEQ_LEN
            num_out: list, the output size of D/B
        """
        super(MetaModel, self).__init__()
        self.vocab_size = vocab_size
        self.emb_dim = emb_dim
        self.seq_len = seq_len
        self.num_out = num_out

        self.context = nn.Sequential(
            nn.Embedding(self.vocab_size[0], self.emb_dim),
            nn.Conv1d(self.seq_len, 64, kernel_size=2),
            nn.ReLU(),
            nn.MaxPool1d(99, 1),
            nn.Flatten(),
        )
        self.AST = nn.Sequential(
            nn.Embedding(self.vocab_size[1], self.emb_dim),
            # nn.LSTM(self.emb_dim, 25, batch_first=True, bidirectional=True),
            # nn.MaxPool1d(50, 1),
            nn.Conv1d(self.seq_len, 50, kernel_size=2),
            nn.ReLU(),
            nn.MaxPool1d(99, 1),
            nn.Flatten(),
        )
        self.fc = nn.Sequential(
            nn.BatchNorm1d(114, affine=False), # one head
            nn.Dropout(0.5),
            nn.Linear(114, 50), # one head
            nn.ReLU()
        )
        self.fc_d = nn.Sequential(
            nn.Linear(50, self.num_out[0]),
            # nn.Sigmoid(),
        )

        self.fc_b = nn.Sequential(
            nn.Linear(50, self.num_out[1]),
            # nn.Sigmoid(),
        )

    def forward(self, x_C, x_A):
        # x, y
        x_C = self.context(x_C) 
        x_A = self.AST(x_A) # one head
        x = torch.concat((x_C, x_A), 1) # one head
        x = self.fc(x) # one head
        y_d = self.fc_d(x) # one tail 
        y_b = self.fc_b(x) # one tail 
        return y_d, y_b # one tail

In [ ]:
N_STEPS = len(train_loader) * EPOCH
def get_lr_scheduler(optimizer):
    return torch.optim.lr_scheduler.OneCycleLR(
        optimizer=optimizer,
        max_lr=Learning_Rate,
        total_steps=N_STEPS,
        pct_start=0.10,
        anneal_strategy='cos',
        div_factor=1e3,
        final_div_factor=1e4,
    )


# ASL LOSS

In [ ]:
class AsymmetricLossOptimized(nn.Module):
    ''' Notice - optimized version, minimizes memory allocation and gpu uploading,
    favors inplace operations'''

    def __init__(self, gamma_neg=4, gamma_pos=1, clip=0.05, eps=1e-8, disable_torch_grad_focal_loss=False):
        super(AsymmetricLossOptimized, self).__init__()

        self.gamma_neg = gamma_neg
        self.gamma_pos = gamma_pos
        self.clip = clip
        self.disable_torch_grad_focal_loss = disable_torch_grad_focal_loss
        self.eps = eps

        # prevent memory allocation and gpu uploading every iteration, and encourages inplace operations
        self.targets = self.anti_targets = self.xs_pos = self.xs_neg = self.asymmetric_w = self.loss = None

    def forward(self, x, y):
        """"
        Parameters
        ----------
        x: input logits
        y: targets (multi-label binarized vector)
        """

        self.targets = y
        self.anti_targets = 1 - y

        # Calculating Probabilities
        self.xs_pos = torch.sigmoid(x)
        self.xs_neg = 1.0 - self.xs_pos

        # Asymmetric Clipping
        if self.clip is not None and self.clip > 0:
            self.xs_neg.add_(self.clip).clamp_(max=1)

        # Basic CE calculation
        self.loss = self.targets * torch.log(self.xs_pos.clamp(min=self.eps))
        self.loss.add_(self.anti_targets * torch.log(self.xs_neg.clamp(min=self.eps)))

        # Asymmetric Focusing
        if self.gamma_neg > 0 or self.gamma_pos > 0:
            if self.disable_torch_grad_focal_loss:
                torch.set_grad_enabled(False)
            self.xs_pos = self.xs_pos * self.targets
            self.xs_neg = self.xs_neg * self.anti_targets
            self.asymmetric_w = torch.pow(1 - self.xs_pos - self.xs_neg,
                                          self.gamma_pos * self.targets + self.gamma_neg * self.anti_targets)
            if self.disable_torch_grad_focal_loss:
                torch.set_grad_enabled(True)
            self.loss *= self.asymmetric_w

        return -self.loss.sum()

In [ ]:
# model
model = MetaModel(vocab_size, EMB_DIM, MAX_SEQ_LEN, num_out).to(device)

# loss_func
# def loss_fn(y, y_pred):
#     return -torch.sum(0.8*y*torch.log(y_pred) + 0.2*(1-y)*torch.log(1-y_pred))
# loss_fn = torch.nn.BCEWithLogitsLoss().to(device)
# loss_fn_mse = torch.nn.MSELoss().to(device)
# loss_fn_ce = torch.nn.MultiLabelSoftMarginLoss().to(device)
# loss_fn_ce = torch.nn.BCEWithLogitsLoss().to(device)
# loss_name = 'BCELogits_Adam'
# loss_name = 'BCELogits'
loss_name = 'AsymmetricLossOptimized'
loss_fn = AsymmetricLossOptimized().to(device)
# Optimizer
optimizer = torch.optim.Adam(
    params=model.parameters(),
    lr=Learning_Rate,
)

# Learning Rate Scheduler
LR_SCHEDULER = get_lr_scheduler(optimizer)

In [ ]:
def metrics(y: torch.Tensor, y_pred: torch.Tensor, threshold: float = 0.5, from_logits = False, split_pos = num_out):
    if from_logits == True:
        y_pred = nn.Sigmoid()(y_pred)
    pred = torch.where(y_pred > threshold, 1, 0)
    
    y_d, y_b = torch.split(y, split_pos, dim=1)
    pred_d, pred_b = torch.split(pred, split_pos, dim=1)
    
    TPd, TPb = torch.sum(y_d * pred_d, dim=1), torch.sum(y_b * pred_b, dim=1)
    TNd, TNb = torch.sum((1-y_d) * (1-pred_d), dim=1), torch.sum((1-y_b) * (1-pred_b), dim=1)
    FPd, FPb = torch.sum((1-y_d) * pred_d , dim=1), torch.sum((1-y_b) * pred_b , dim=1)
    FNd, FNb = torch.sum(y_d * (1-pred_d), dim=1), torch.sum(y_b * (1-pred_b), dim=1)

    acc = torch.mean((TPd + TNd) / (TPd + TNd + FPd + FNd + 1e-6)).item(), torch.mean((TPb + TNb) / (TPb + TNb + FPb + FNb + 1e-6)).item()
    recall = torch.mean(TPd / (TPd + FNd + 1e-6)).item(), torch.mean(TPb / (TPb + FNb + 1e-6)).item()
    precision = torch.mean(TPd / (TPd + FPd + 1e-6)).item(), torch.mean(TPb / (TPb + FPb + 1e-6)).item()
    F1 = 2 * recall[0] * precision[0] / (recall[0] + precision[0] + 1e-6), 2 * recall[1] * precision[1] / (recall[1] + precision[1] + 1e-6)
    
    return {
        'acc': acc,
        'precision': precision,
        'recall': recall,
        'F1': F1
    }

def one_forward(data):
    (x_context, x_AST), (y_dev, y_btype) = data
    y_dev_pred, y_btype_pred = model(x_context.long().to(device), x_AST.long().to(device)) 
    
    y = torch.concat((y_dev, y_btype), 1).to(device) 
    y_pred = torch.concat((y_dev_pred, y_btype_pred), 1) 

    # loss = Weight_MSE * loss_fn_mse(y, nn.Sigmoid()(y_pred)) + Weight_BCE * loss_fn_ce(y, y_pred)
    loss = loss_fn(y_pred, y)
    metric = metrics(y, y_pred, from_logits=False)
    
    return loss, metric['acc'], metric['precision'], metric['recall'], metric['F1']


def one_backward(optimizer, loss):
    optimizer.zero_grad()
    loss.backward()
    # Gradient clipping
    torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
    optimizer.step()

def one_train_scaler(scaler, data, optimizer):
    with torch.cuda.amp.autocast():
        optimizer.zero_grad()
        loss, acc, precision, recall, f1 = one_forward(data)
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

In [ ]:
print(num_out)

In [30]:
from torch.utils.tensorboard import SummaryWriter
# torch.autograd.set_detect_anomaly(True)
# scaler = torch.cuda.amp.GradScaler()

writer = SummaryWriter('./tf-logs')
for epoch in tqdm.trange(EPOCH):

    # train
    model.train()
    for step, data in enumerate(train_loader):

        loss, acc, precision, recall, f1 = one_forward(data)
        one_backward(optimizer, loss)
        
        # one_train_scaler(scaler, data, optimizer)
        # LR_SCHEDULER.step()
        

    # val
    model.eval()
    val_loss, val_acc, val_precision, val_recall, val_F1 = 0, [0, 0], [0, 0], [0, 0], [0, 0]
    with torch.no_grad():
        for step, data in enumerate(test_loader):
            loss, acc, precision, recall, f1 = one_forward(data)
            l = len(test_loader)
            val_loss += loss.item() / l
            val_acc[0] += acc[0]/l
            val_acc[1] += acc[1]/l
            val_precision[0] += precision[0]/l
            val_precision[1] += precision[1]/l
            val_recall[0] += recall[0]/l
            val_recall[1] += recall[1]/l
            val_F1[0] += f1[0]/l 
            val_F1[1] += f1[1]/l 
            
    print('{}th epoch\n val_loss: {}\n val_acc:{}\n val_precision:{}\n val_recall:{}\n val_f1: {}'.format(epoch, val_loss, val_acc, val_precision, val_recall, val_F1))
    writer.add_scalar('val_loss'+loss_name+str(Learning_Rate), val_loss, epoch)
    writer.add_scalar('val_acc_d'+loss_name+str(Learning_Rate), val_acc[0], epoch)
    writer.add_scalar('val_acc_b'+loss_name+str(Learning_Rate), val_acc[1], epoch)
    # writer.add_scalar('val_precision'+loss_name+str(Learning_Rate), val_precision, epoch)
    # writer.add_scalar('val_recall'+loss_name+str(Learning_Rate), val_recall, epoch)
    writer.add_scalar('val_f1_d'+loss_name+str(Learning_Rate), val_F1[0], epoch)
    writer.add_scalar('val_f1_b'+loss_name+str(Learning_Rate), val_F1[1], epoch)

    # save model
    # if epoch % 20 == 0:
        # torch.save(model, '../savedmodel/model_epoch{}.pth'.format(epoch))

writer.close()

  2%|▏         | 6/300 [00:06<05:21,  1.09s/it]

5th epoch
 val_loss: 507.82591181216037
 val_acc:[0.9776154160499573, 0.9691654728806537]
 val_precision:[0.0009057967032751312, 0.014577137255717233]
 val_recall:[0.0020380415346311484, 0.020783918018898225]
 val_f1: [0.0012454153248318742, 0.015810463057772728]


  2%|▏         | 7/300 [00:07<05:18,  1.09s/it]

6th epoch
 val_loss: 502.58728027343744
 val_acc:[0.9797378519307012, 0.9714663495188174]
 val_precision:[0.0013586946803590524, 0.014912711871463966]
 val_recall:[0.0020380415346311484, 0.01676444478494965]
 val_f1: [0.0015850833196342671, 0.01375202081851601]


  3%|▎         | 8/300 [00:08<05:16,  1.08s/it]

...

297th epoch
 val_loss: 193.93571239969003
 val_acc:[0.9901342132817144, 0.9833854566449705]
 val_precision:[0.5717632990816365, 0.0020380415346311484]
 val_recall:[0.5717632990816365, 0.000849184418178123]
 val_f1: [0.5717627990820808, 0.00117748299429606]


100%|█████████▉| 299/300 [05:33<00:01,  1.15s/it]

298th epoch
 val_loss: 193.6730668441109
 val_acc:[0.990115077599235, 0.9833931767422219]
 val_precision:[0.5704046034294626, 0.0020380415346311484]
 val_recall:[0.5704046034294626, 0.000849184418178123]
 val_f1: [0.5704041034299078, 0.00117748299429606]


100%|██████████| 300/300 [05:34<00:00,  1.12s/it]

299th epoch
 val_loss: 193.74813643745753
 val_acc:[0.9901724872381791, 0.9833854540534641]
 val_precision:[0.574480687794478, 0.0027173887128415313]
 val_recall:[0.574480687794478, 0.001528531596388506]
 val_f1: [0.5744801877949199, 0.0018891780277030474]


In [31]:
(x_c, x_a), (y_d, y_b) = test_dataset[0]
x_c = torch.reshape(x_c.long().to(device), (1, 300))
x_a = torch.reshape(x_a.long().to(device), (1, 300))
y_d_pred, y_b_pred = model(x_c, x_a)

print(y_d_pred, y_b_pred)
# y_d_pred, y_b_pred = nn.Softmax(dim=1)(y_d_pred), nn.Sigmoid()(y_b_pred)
print(y_d_pred[0][-29])
#TODO: 调整阈值
#TODO: 多标签分类问题：换用F1分数、Hamming损失或Jaccard相似度
print(torch.where(y_d_pred > 0.5, 1.0, 0.0), torch.where(y_b_pred > 0.5, 1.0, 0.0))


tensor([[-0.5192, -0.7102, -0.5605, -0.6330, -0.5789, -1.1756, -0.6486, -0.2284,
         -1.1637, -1.2900, -0.9660, -1.2959, -0.8545, -1.3448, -0.9044, -1.1265,
         -0.5622, -0.9416, -0.8007, -0.8651, -0.1522, -0.6373, -0.8191, -1.5373,
         -1.1547, -0.6090, -1.6119, -1.0397, -1.1826, -0.2061, -0.4938, -0.9570,
          0.0642, -0.5006, -0.6189, -0.8170, -1.5933, -0.4129, -0.8423, -0.4953,
         -0.7961, -0.4890, -0.3032, -1.2003, -0.1174, -0.8870, -0.3913, -0.1565,
         -0.5866, -0.0068, -0.5953, -0.6872, -0.7338, -1.3384, -0.3298, -0.5983,
         -1.2355, -0.9516, -1.3758, -1.2975, -0.3335, -0.6822, -0.4234, -0.5415,
         -1.2079, -0.4500, -0.7568, -1.1634,  0.3400, -0.5799, -0.9790]],
       grad_fn=<AddmmBackward0>) tensor([[-6.3927e-01, -4.6033e-01, -1.3159e+00, -1.5771e+00, -1.1543e+00,
         -4.1967e-01, -8.2346e-01, -3.5303e-01, -7.3370e-01, -4.9470e-01,
         -8.8097e-01, -7.3276e-01, -6.6635e-01, -1.1465e+00, -1.1102e+00,
         -4.5608e-01, -

In [32]:
print(y_d, y_b)

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]) tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0.,

In [33]:
# y_d = y_d.reshape(1, -1)
# y_d_pred = y_d_pred.reshape(1, -1)
# for i, l in metrics(y_d.to(device), y_d_pred, from_logits = True, split_pos = num_out[0]).items():
#     print(i, l)

In [34]:
# y_b = y_b.reshape(1, -1)
# y_b_pred = y_b_pred.reshape(1, -1)
# metrics(y_b.to(device), y_b_pred, from_logits = True), metrics(y_b.to(device), y_b_pred, from_logits = False), metricsy_b.to(device), y_b_pred, from_logits = True, recall=True), metricsy_b.to(device), y_b_pred, from_logits = False, recall=True)

In [35]:
print(num_out)

[71, 176]
